# Extraction Notebook

In [2]:
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

## Testing and model config

In [4]:
chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

In [5]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content='저는 파이썬을 사랑합니다.\n\nTranslation:\nI love Python.', response_metadata={'id': 'msg_01YQxnA6usGmYdERQS12DZc7', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 22, 'output_tokens': 28}}, id='run-5b715f5c-189c-4222-95f1-6a6ba9bf67ec-0')

## Extraction Chain

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value."
            "before you formulate your response think about who the individual speakers are and enclose your thoughts in thinking tags"
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [26]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person. This field is required.")
    image_url: Optional[str] = Field('', description="URL of an image of the person. This is usually left empty.")
    facts: List[str] = Field(default_factory=list, description="A list of miscellaneous facts about the person.")

In [31]:
text = """
"Hey man, how\'s it going? Good, good. How about you? '
 "What's your favorite animal, man? Pretty good. My favorite animal is "
 "dinosaur. But let me start, like, what's your name? I'm Arush. What's yours? "
 "I'm Vig, by the way. Nice to meet you. So how do you like the conference? "
 "It's pretty cool. It's really big. I'm meeting a lot of really cool people. "
 "My favorite animal is the capybara, so I've been finding a lot of capybara "
 "lovers around here. It's been nice. Tell me more about capybara. They're "
 'just really goofy. Interesting. Is that why you like them? Yeah, yeah. '
 "It's... That's very interesting, man. Like, I've never seen someone who's a "
 "fan of capybara. It's the first time. Yeah, they're like huge rats, "
 "basically. Oh, okay. Interesting. They're really cute, and there's been a "
 'trend going on, going around on, like, Instagram and stuff of reels of '
 "capybaras doing random stuff. They're like pandas, by the way. That's "
 'amazing. It kind of reminds me of how, you know, people use animal names '
 "for, like, technology stuff. Like, you know, I don't want to quite say LLM, "
 "but, you know, llama is something. Yeah. So, but that's very interesting. "
 "I'm a huge fan of dinosaur, and one of the popular things is just, I think "
 "they're very cool. I used to be a very much a fan of a velociraptor, which "
 'is like, you know, a dinosaur, right? Yeah. What really sparked your '
 "interest in velociraptors? Was it Jurassic Park? I could, it's partly the "
 "Jurassic Park, but I would say it's more like the way they behave. Something "
 "that was very attractive is just like, you know, they're very intelligent. "
 "They work as a group, and, you know, they're kind of excited, like, it's "
 "kind of like they're not very big, but they're like super genius, right? "
 'Yeah. Like the dog, yes, but, all right. I hope you have a good conference, '
 'man. Yeah, you too, man. I\'ll see you around. See ya."

"""

In [34]:
text

'\n"Hey man, how\'s it going? Good, good. How about you? \'\n "What\'s your favorite animal, man? Pretty good. My favorite animal is "\n "dinosaur. But let me start, like, what\'s your name? I\'m Arush. What\'s yours? "\n "I\'m Vig, by the way. Nice to meet you. So how do you like the conference? "\n "It\'s pretty cool. It\'s really big. I\'m meeting a lot of really cool people. "\n "My favorite animal is the capybara, so I\'ve been finding a lot of capybara "\n "lovers around here. It\'s been nice. Tell me more about capybara. They\'re "\n \'just really goofy. Interesting. Is that why you like them? Yeah, yeah. \'\n "It\'s... That\'s very interesting, man. Like, I\'ve never seen someone who\'s a "\n "fan of capybara. It\'s the first time. Yeah, they\'re like huge rats, "\n "basically. Oh, okay. Interesting. They\'re really cute, and there\'s been a "\n \'trend going on, going around on, like, Instagram and stuff of reels of \'\n "capybaras doing random stuff. They\'re like pandas, by 

In [28]:
# runnable = prompt | chat.with_structured_output(schema=Person)

# runnable.invoke({"text": text, "input_language": "English",
#         "output_language": "English",})


In [32]:
class Conversation(BaseModel):
    """Model for capturing conversation details.

    This class captures the essentials of a conversation, including who was involved, when it occurred, and a summary of the discussion. The timestamp is optional and can be left empty, aligning with the requirement to occasionally omit the specific time of the conversation.
    """

    persons: List[Person]
    timestamp: Optional[str] = Field(None, description="The timestamp when the conversation occurred. This field is always empty.")
    summary:Optional[str] = Field(..., description="A bulleted summary list of key points discussed in the conversation.")

In [33]:
runnable = prompt | chat.with_structured_output(schema=Conversation)

runnable.invoke({"text": text})


Conversation(persons=[Person(name='Arush', image_url='', facts=["Arush's favorite dinosaur is the velociraptor", 'Arush likes velociraptors for their intelligence and group behavior', 'Arush was partly inspired by Jurassic Park for his interest in velociraptors']), Person(name='Vig', image_url='', facts=["Vig's favorite animal is the capybara", 'Vig finds capybaras goofy and cute', 'Vig has seen Instagram reels of capybaras doing random things'])], timestamp='null', summary="- Arush and Vig meet for the first time at a conference\n- Vig's favorite animal is the capybara, which he finds goofy and cute based on Instagram reels\n- Arush's favorite dinosaur is the velociraptor, which he likes for their intelligence and group behavior, partly inspired by Jurassic Park \n- They wish each other well at the end of the conversation")

## Preprocess transcript

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240229")

system = (
    "You are a helpful assistant that processes transcripts of conversations. You always identify speakers and return a diarized transcripts with names.
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])


In [36]:
class Segment(BaseModel):
    """One segment of a conversation."""

    text: str = Field(..., description="The segment of the transcript. This field is required.")
    speaker_name: str = Field(..., description="The name of the speaker. This field is required.")

class Transcript(BaseModel):
    """A transcript of a conversation."""

    segments: List[Segment] = Field(..., description="A list of segments in the transcript. This field is required.")

In [37]:
runnable = prompt | chat.with_structured_output(schema=Transcript)

runnable.invoke({"text": text})

Transcript(segments=[Segment(text="Hey man, how's it going? Good, good. How about you?", speaker_name='Arush'), Segment(text="What's your favorite animal, man? Pretty good. My favorite animal is dinosaur. But let me start, like, what's your name? I'm Arush. What's yours?", speaker_name='Vig'), Segment(text="I'm Vig, by the way. Nice to meet you. So how do you like the conference?", speaker_name='Arush'), Segment(text="It's pretty cool. It's really big. I'm meeting a lot of really cool people. My favorite animal is the capybara, so I've been finding a lot of capybara lovers around here. It's been nice.", speaker_name='Vig'), Segment(text="Tell me more about capybara. They're just really goofy. Interesting. Is that why you like them?", speaker_name='Arush'), Segment(text="Yeah, yeah. It's... That's very interesting, man. Like, I've never seen someone who's a fan of capybara. It's the first time.", speaker_name='Vig'), Segment(text="Yeah, they're like huge rats, basically. Oh, okay. Inter